<a href="https://colab.research.google.com/github/japerego/DroidCafe/blob/master/Proyecto_3_4_Dise%C3%B1o_Avanzado_Otto_ENSEMBLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- *Alumno*:
- *Asignatura*: Ingeniería del Conocimiento
- *Curso*: 2021/2022
- *Profesor*: Fco. Javier Martínez de Pisón
- Universidad de La Rioja

# Proyecto 3. Parte 4. Combinación Ponderada de Modelos

En esta parte vamos a aprender a realizar una combinación ponderada de modelos *weigthed ensemble models".

In [25]:
!pip install bayesian-optimization
!pip install lightgbm
!pip install xgboost --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
# Importamos paquetes basicos
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
# from tqdm.notebook import tqdm
from tqdm.notebook import tqdm

import sklearn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier 
from sklearn.metrics import log_loss
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost.sklearn import XGBClassifier
from lightgbm.sklearn import LGBMClassifier

from bayes_opt import BayesianOptimization

# %matplotlib inline
plt.style.use('ggplot')
# Elimina la limitación en visualización del numero de columnas
pd.set_option('display.max_columns', None) 

## Cargamos y Preprocesamos la Base de Datos

In [27]:
!wget https://github.com/IC1920/Datasets/raw/master/train.zip -O train.zip
!wget https://github.com/IC1920/Datasets/raw/master/test.zip -O test.zip
!unzip -o train.zip
!unzip -o test.zip

--2022-05-30 15:37:52--  https://github.com/IC1920/Datasets/raw/master/train.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IC1920/Datasets/master/train.zip [following]
--2022-05-30 15:37:52--  https://raw.githubusercontent.com/IC1920/Datasets/master/train.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1655239 (1.6M) [application/zip]
Saving to: ‘train.zip’

train.zip           100%[===================>]   1.58M  --.-KB/s    in 0.06s   

2022-05-30 15:37:52 (27.8 MB/s) - ‘train.zip’ saved [1655239/1655239]

--2022-05-30 15:37:52--  https://github.com/IC1920/Datasets/raw/master/test.zip

In [28]:
# Leemos los archivos arff
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
print(df_train.shape, df_test.shape)

features = ['feat_'+str(i) for i in np.arange(1,94)]
target = 'target'

(61878, 95) (144368, 94)


In [29]:
scaler = StandardScaler()
X_train = df_train[features].values.copy()
X_test = df_test[features].values.copy()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

Preparamos las bases de datos. Para poder hacer una validación local, extraemos un 20% de forma aleatoria de forma estratificada.

In [30]:
X = X_train_scaled
y = df_train[target].values.reshape(-1)
y = LabelEncoder().fit_transform(y)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=36, stratify=y)

print("Verificamos que la partición es estratificada")
print(np.round(np.bincount(y_train)/len(y_train),3))
print(np.round(np.bincount(y_val)/len(y_val),3))

Verificamos que la partición es estratificada
[0.031 0.261 0.129 0.043 0.044 0.228 0.046 0.137 0.08 ]
[0.031 0.261 0.129 0.043 0.044 0.228 0.046 0.137 0.08 ]


## Combinación Ponderada de Modelos

Vamos a intentar afinar usando una combinación ponderada de los tres modelos.

Para poder combinar los modelos y aprovechar al máximo la base de datos, desarrollamos una función que entrena los modelos con validación cruzada estratificada y nos devuelve la predicción de validación, además de la predicción con la base de datos de testeo.

In [31]:
# Función que obtiene el LogLoss con repetición y para la base de datos de testeo
def entrena_model_folds(model, num_folds=5, num_bags=1, calibra=True):
    np.random.seed(2020)
    X_tot = X.copy()
    y_tot = y.copy()
    X_test = X_test_scaled.copy()
    
    # Creamos arrays para las predicciones
    preds_val = np.empty((len(X_tot), 9, num_bags))
    preds_val[:] = np.nan
    preds_test = np.empty((len(X_test), 9, num_bags*num_folds))
    preds_test[:] = np.nan
    # Entrena y extrae la predicciones con validación cruzada repetida
    
    folds = RepeatedStratifiedKFold(n_splits=num_folds, n_repeats=num_bags, random_state=2020)  
    for niter, (train_index, val_index) in enumerate(folds.split(X_tot, y_tot)):
        print('ITER=',niter)
        nbag = niter//num_folds #Extrae el número de repetición (bag)
        X_train, X_val = X_tot[train_index], X_tot[val_index]
        y_train, y_val = y_tot[train_index], y_tot[val_index]
        if calibra:
            calibrated_clf = CalibratedClassifierCV(model, method='isotonic', cv=5)
            calibrated_clf.fit(X_train, y_train)
            preds_val[val_index,:,nbag] = calibrated_clf.predict_proba(X_val) 
            preds_test[:,:,niter] = calibrated_clf.predict_proba(X_test)
        else:
            model.fit(X_train, y_train)
            preds_val[val_index,:,nbag] = model.predict_proba(X_val) 
            preds_test[:,:,niter] = model.predict_proba(X_test)
        
    # Promedia las predicciones
    preds_val_mean = preds_val.mean(axis=2) 
    preds_test_mean = preds_test.mean(axis=2)

    # Extrae las métricas
    log_loss_val = log_loss(y_tot, preds_val_mean, eps=1e-15, normalize=True)
    
    return log_loss_val, y_tot, preds_val_mean, preds_test_mean

## Extraemos las predicciones con Validación Cruzada 5

**ATENCION: El coste computacional del calculo de los modelos de este apartado es elevado. NO EJECUTAR EL CALCULO SI NO SE DISPONE DE UN EQUIPO LOCAL DE ALTAS PRESTACIONES.**

Definimos los modelos con los parámetros encontrados en la etapa anterior.

In [32]:
CREA_MODELOS = False

if CREA_MODELOS:
    rf_model = RandomForestClassifier(n_estimators=600,
                                      max_features=9,
                                      min_samples_split=2,
                                      n_jobs=-1,class_weight='balanced')

    logistic_model = LogisticRegression(C=2.0, max_iter=450, n_jobs=-1,class_weight=None,random_state=42)

    xgb_model = XGBClassifier(learning_rate =0.1, n_estimators=650, max_depth=8, 
                              subsample=0.8, colsample_bytree=0.6, nthread=20,
                              reg_alpha=0.001, reg_lambda=1.0,
                              seed=42, objective='multi:softprob')

In [33]:
%%time
if CREA_MODELOS:
    loss_rf, y_rf, preds_val_rf, preds_test_rf  = entrena_model_folds(rf_model, calibra=True)
    print('RF=',loss_rf)
    loss_logit, y_logit, preds_val_logit, preds_test_logit = entrena_model_folds(logistic_model, calibra=False)
    print('LOGIT=',loss_logit)
    loss_xgb, y_xgb, preds_val_xgb, preds_test_xgb = entrena_model_folds(xgb_model, calibra=True)
    print('XGB=',loss_xgb)

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 11.4 µs


## Leemos las predicciones de los modelos

In [34]:
!wget https://github.com/IC1920/Datasets/raw/master/preds_rf_logit_xgb.zip -O preds_rf_logit_xgb.zip
!wget https://github.com/IC1920/Datasets/raw/master/preds_rf_logit_xgb.z01 -O preds_rf_logit_xgb.z01
!7z x preds_rf_logit_xgb.zip -y

--2022-05-30 15:37:56--  https://github.com/IC1920/Datasets/raw/master/preds_rf_logit_xgb.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IC1920/Datasets/master/preds_rf_logit_xgb.zip [following]
--2022-05-30 15:37:56--  https://raw.githubusercontent.com/IC1920/Datasets/master/preds_rf_logit_xgb.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13443045 (13M) [application/zip]
Saving to: ‘preds_rf_logit_xgb.zip’

preds_rf_logit_xgb. 100%[===================>]  12.82M  --.-KB/s    in 0.1s    

2022-05-30 15:37:56 (121 MB/s) - ‘preds_rf_logit_xgb.zip’ saved [13443045/13443045]

--2022-05-30 

In [35]:
import pickle
if False: #Grabamos o leemos
    file = open('preds_rf_logit_xgb.pickle','wb')
    pickle.dump(preds_val_rf, file, protocol=4)
    pickle.dump(preds_test_rf, file, protocol=4)
    pickle.dump(preds_val_logit, file, protocol=4)
    pickle.dump(preds_test_logit, file, protocol=4)
    pickle.dump(preds_val_xgb, file, protocol=4)
    pickle.dump(preds_test_xgb, file, protocol=4)
    pickle.dump(y_xgb, file, protocol=4) # Los tres y son iguales
    file.close()
else:
    file = open('preds_rf_logit_xgb.pickle','rb')
    preds_val_rf = pickle.load(file)
    preds_test_rf = pickle.load(file)
    preds_val_logit = pickle.load(file)
    preds_test_logit = pickle.load(file)
    preds_val_xgb = pickle.load(file)
    preds_test_xgb = pickle.load(file)
    y_xgb = pickle.load(file)
    file.close()

Vamos a crear las *submissions* con la media de la predicción de los modelos obtenidos en cada uno de los 5Folds

In [36]:
output = pd.DataFrame(preds_test_rf,columns=["Class_"+str(i) for i in range(1,10)])
output.insert(loc=0, column='id', value=df_test.id)
output.to_csv('submission_rf_5cv.csv', index=False)
pd.read_csv('submission_rf_5cv.csv').head()

,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,1,0.000770,0.130241,0.193129,0.664083,0.000022,0.002050,0.008825,0.000532,0.000348
1,2,0.005616,0.002344,0.000949,0.003024,0.000319,0.723761,0.002611,0.259850,0.001527
2,3,0.000145,0.000018,0.000018,0.000269,0.000016,0.998147,0.000222,0.001121,0.000044
3,4,0.000702,0.576385,0.365200,0.031083,0.000040,0.000244,0.001135,0.000815,0.024397
4,5,0.108842,0.000014,0.000016,0.000178,0.000021,0.005742,0.002357,0.245595,0.637235


Este modelo RF obtiene con 5CV un **Local=0.48133, Private_LB=0.47482 Public_LB=0.47200**

In [37]:
output = pd.DataFrame(preds_test_logit,columns=["Class_"+str(i) for i in range(1,10)])
output.insert(loc=0, column='id', value=df_test.id)
output.to_csv('submission_logit_5cv.csv', index=False)
pd.read_csv('submission_logit_5cv.csv').head()

,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,1,1.013086e-05,1.571883e-01,2.503112e-01,5.880452e-01,9.971093e-14,8.684473e-06,4.436067e-03,3.202108e-07,1.859623e-08
1,2,4.279658e-04,4.100173e-05,2.051091e-07,1.769082e-08,2.268213e-06,6.715051e-01,2.213332e-04,3.264076e-01,1.394516e-03
2,3,3.120890e-06,2.469223e-08,7.062547e-09,2.515388e-11,2.101699e-10,9.998786e-01,5.060433e-05,6.682249e-05,8.082613e-07
3,4,7.268450e-10,7.117819e-01,2.815075e-01,6.710282e-03,1.908572e-09,1.191905e-08,9.950225e-08,3.263994e-09,2.120376e-07
4,5,6.610212e-01,2.647432e-07,7.054207e-08,2.738912e-09,1.678788e-15,4.869883e-03,1.209282e-03,3.923410e-02,2.936652e-01


Este modelo LOGIT obtiene con 5CV un **Local=0.64006, Private_LB=0.63270, Public_LB=0.63214**.

In [38]:
output = pd.DataFrame(preds_test_xgb,columns=["Class_"+str(i) for i in range(1,10)])
output.insert(loc=0, column='id', value=df_test.id)
output.to_csv('submission_xgb_5cv.csv', index=False)
pd.read_csv('submission_xgb_5cv.csv').head()

,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,1,0.000338,0.166959,0.177536,0.650979,0.000000,0.000104,0.003725,0.000344,0.000015
1,2,0.002120,0.007067,0.001531,0.002146,0.000257,0.435691,0.007227,0.542036,0.001926
2,3,0.000000,0.000000,0.000015,0.000016,0.000000,0.999746,0.000000,0.000224,0.000000
3,4,0.000197,0.655010,0.329998,0.008723,0.000000,0.000013,0.000294,0.000166,0.005599
4,5,0.206780,0.000237,0.000450,0.000142,0.000071,0.009465,0.002476,0.074920,0.705457


Este modelo XGB obtiene con 5CV un **Local=0.45116, Private_LB=0.43628, Public_LB=0.43618**. Un modelo muy bueno que **nos situaria en la posición 428 del Private Leaderboard de 3511 equipos (percentil 87.80)**. 

## Buscamos los pesos que minimizan el error

In [39]:
def combina_preds3(w):
    w1 = w[0]
    w2 = w[1]
    w3= 3.0-w1-w2
    preds_combi = (w1*preds_val_rf+w2*preds_val_logit+w3*preds_val_xgb)/3.0
    return log_loss(y_xgb, preds_combi, eps=1e-15, normalize=True)    

In [40]:
combina_preds3([0.6,0.1])

0.4457511028745261

In [41]:
from scipy.optimize import minimize
res = minimize(combina_preds3, [0.6, 0.1], method='nelder-mead',options={'xatol': 1e-8, 'disp': True})
print(res)

Optimization terminated successfully.
         Current function value: 0.444678
         Iterations: 67
         Function evaluations: 139
 final_simplex: (array([[0.55249128, 0.00227782],
       [0.55249129, 0.00227782],
       [0.55249129, 0.00227782]]), array([0.44467763, 0.44467763, 0.44467763]))
           fun: 0.4446776340060137
       message: 'Optimization terminated successfully.'
          nfev: 139
           nit: 67
        status: 0
       success: True
             x: array([0.55249128, 0.00227782])


El peso del modelo LOGIT es muy pequeño. Vamos a probar solo con el RF y XGB.

In [42]:
def combina_preds2(w):
    w1 = w[0]
    w2= 2.0-w1
    preds_combi = (w1*preds_val_rf+w2*preds_val_xgb)/2.0
    return log_loss(y_xgb, preds_combi, eps=1e-15, normalize=True)    

In [43]:
from scipy.optimize import minimize
res = minimize(combina_preds2, [0.8], method='nelder-mead',options={'xatol': 1e-8, 'disp': True})
print(res)

Optimization terminated successfully.
         Current function value: 0.444957
         Iterations: 28
         Function evaluations: 59
 final_simplex: (array([[0.36925201],
       [0.36925202]]), array([0.44495652, 0.44495652]))
           fun: 0.4449565166066821
       message: 'Optimization terminated successfully.'
          nfev: 59
           nit: 28
        status: 0
       success: True
             x: array([0.36925201])


Con tres el resultado es ligeramente mejor que con dos: 0.4446776340060137 vs 0.4449565166066821

## Creamos la submission con la combinación de los tres modelos

In [44]:
res = minimize(combina_preds3, [0.6, 0.1], method='nelder-mead',options={'xatol': 1e-8, 'disp': True})
print(res)

w1 = res.x[0]
w2 = res.x[1]
w3 = 3.0-w1-w2

preds_combi_tst = (w1*preds_test_rf+w2*preds_test_logit+w3*preds_test_xgb)/3.0
output = pd.DataFrame(preds_combi_tst,columns=["Class_"+str(i) for i in range(1,10)])
output.insert(loc=0, column='id', value=df_test.id)
output.to_csv('submission_combi_5cv.csv', index=False)
pd.read_csv('submission_combi_5cv.csv').head()

Optimization terminated successfully.
         Current function value: 0.444678
         Iterations: 67
         Function evaluations: 139
 final_simplex: (array([[0.55249128, 0.00227782],
       [0.55249129, 0.00227782],
       [0.55249129, 0.00227782]]), array([0.44467763, 0.44467763, 0.44467763]))
           fun: 0.4446776340060137
       message: 'Optimization terminated successfully.'
          nfev: 139
           nit: 67
        status: 0
       success: True
             x: array([0.55249128, 0.00227782])


,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,1,0.000417,0.160189,0.180463,0.653345,0.000004,0.000462,0.004665,0.000379,0.000077
1,2,0.002762,0.006192,0.001422,0.002306,0.000268,0.488922,0.006371,0.489903,0.001852
2,3,0.000027,0.000003,0.000015,0.000062,0.000003,0.999451,0.000041,0.000389,0.000008
3,4,0.000290,0.640573,0.336444,0.012840,0.000007,0.000055,0.000448,0.000286,0.009056
4,5,0.189088,0.000196,0.000370,0.000149,0.000062,0.008776,0.002453,0.106325,0.692581


La combinación de los tres modelos obtiene un **Local=0.44468, Private=0.43486, Public=0.43475** que nos situa en en la posición 407 de 3511 (percentil 88.40%) del *private leaderboard* . Este resultado es muy bueno considerando que SE HAN UTILIZADO UNICAMENTE LAS FEATURES ORIGINALES sin preprocesar y sin filtrar.

### EJERCICIO  6

**Añadir las predicciones del modelo LGB para crear un modelo Ensemble buscando los pesos óptimos para obtener la mejor combinación de los modelos (RF, LOGIT, XGB y LGB). NO es obligatorio usar los cuatro modelos pues se puede desestimar alguno de ellos si es necesario. El objetivo es conseguir un modelo Ensemble que minimice lo más posible el LogLoss.**

**Crear la submission y presentar los Scores Finales Publicos y Privados. Incluir en este script una foto de la pantalla de Kaggle con los resultados de la submission y en que posición del PRIVATE LEADERBOARD quedaria.**

In [45]:
modellgbm = LGBMClassifier(learning_rate =0.2, n_estimators=700, max_depth=7, 
                          subsample=0.9, colsample_bytree=0.7, nthread=4,
                          reg_alpha=0.0001, reg_lambda=0.01,subsample_freq= 1, min_child_weight=0.5,
                          seed=42, objective='multi:softprob')
rf_model = RandomForestClassifier(n_estimators=600,max_features=9,min_samples_split=2,n_jobs=-1,class_weight='balanced')
logistic_model = LogisticRegression(C=2.0, max_iter=450, n_jobs=-1,class_weight=None,random_state=42)
xgb_model = XGBClassifier(learning_rate =0.1, n_estimators=650, max_depth=8, subsample=0.8, colsample_bytree=0.6, nthread=20,reg_alpha=0.001, reg_lambda=1.0,seed=42, objective='multi:softprob')

In [ ]:
CREA_MODELOS = False

if CREA_MODELOS:
    rf_model = RandomForestClassifier(n_estimators=600,
                                      max_features=9,
                                      min_samples_split=2,
                                      n_jobs=-1,class_weight='balanced')

    logistic_model = LogisticRegression(C=2.0, max_iter=450, n_jobs=-1,class_weight=None,random_state=42)

    xgb_model = XGBClassifier(learning_rate =0.1, n_estimators=650, max_depth=8, 
                              subsample=0.8, colsample_bytree=0.6, nthread=20,
                              reg_alpha=0.001, reg_lambda=1.0,
                              seed=42, objective='multi:softprob')

In [ ]:
%%time
if CREA_MODELOS:
    loss_rf, y_rf, preds_val_rf, preds_test_rf  = entrena_model_folds(rf_model, calibra=True)
    print('RF=',loss_rf)
    loss_logit, y_logit, preds_val_logit, preds_test_logit = entrena_model_folds(logistic_model, calibra=False)
    print('LOGIT=',loss_logit)
    loss_xgb, y_xgb, preds_val_xgb, preds_test_xgb = entrena_model_folds(xgb_model, calibra=True)
    print('XGB=',loss_xgb)

ITER= 0
ITER= 1
ITER= 2
ITER= 3
ITER= 4
RF= 0.4813305167410231
ITER= 0
ITER= 1
ITER= 2
ITER= 3
ITER= 4
LOGIT= 0.6400686984999122
ITER= 0
ITER= 1
ITER= 2
ITER= 3
ITER= 4
XGB= 0.45115528144663847
CPU times: user 12h 17min 5s, sys: 11min 49s, total: 12h 28min 54s
Wall time: 41min 1s


In [46]:
loss_lgb, y_lgb, preds_val_lgb, preds_test_lgb = entrena_model_folds(modellgbm, calibra=True)
print('LGB=',loss_lgb)

ITER= 0
ITER= 1
ITER= 2
ITER= 3
ITER= 4
LGB= 0.4523077542165836


In [47]:
output = pd.DataFrame(preds_test_lgb,columns=["Class_"+str(i) for i in range(1,10)])
output.insert(loc=0, column='id', value=df_test.id)
output.to_csv('submission_lgb_5cv.csv', index=False)
pd.read_csv('submission_lgb_5cv.csv').head()

,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,1,0.000450,0.165142,0.147003,0.683806,0.000000,0.000061,0.003243,0.000295,0.000000
1,2,0.002138,0.011102,0.001227,0.001045,0.000296,0.455674,0.007667,0.519212,0.001637
2,3,0.000000,0.000000,0.000013,0.000058,0.000000,0.999406,0.000019,0.000504,0.000000
3,4,0.000113,0.699952,0.285283,0.008645,0.000011,0.000013,0.000245,0.000224,0.005514
4,5,0.311563,0.000667,0.001045,0.000408,0.000175,0.017559,0.008963,0.107534,0.552085


In [49]:
def combina_predsej6(w):
    w1 = w[0]
    w2 = w[1]
    w3 = w[2]
    w4= 4.0-w1-w2-w3
    preds_combi = (w1*preds_val_rf+w2*preds_val_logit+w3*preds_val_xgb+w4*preds_val_lgb)/4.0
    return log_loss(y_xgb, preds_combi, eps=1e-15, normalize=True)    

In [50]:
combina_predsej6([0.4,0.1,0.2])

0.44763501319388854

In [56]:
from scipy.optimize import minimize
res1 = minimize(combina_predsej6, [0.4,0.1,0.2], method='nelder-mead',options={'xatol': 1e-8, 'disp': True})
print(res1)

Optimization terminated successfully.
         Current function value: 0.443467
         Iterations: 140
         Function evaluations: 269
 final_simplex: (array([[ 0.76139657, -0.06477597,  2.09651298],
       [ 0.76139657, -0.06477597,  2.09651298],
       [ 0.76139657, -0.06477597,  2.09651298],
       [ 0.76139657, -0.06477597,  2.09651298]]), array([0.44346735, 0.44346735, 0.44346735, 0.44346735]))
           fun: 0.44346735101107543
       message: 'Optimization terminated successfully.'
          nfev: 269
           nit: 140
        status: 0
       success: True
             x: array([ 0.76139657, -0.06477597,  2.09651298])


El peso del modelo logistico es muy pequeño, lo eliminamos

In [52]:
def combina_predsej6_2(w):
    w1 = w[0]
    w2 = w[1]
    w3= 3.0-w1-w2
    preds_combi = (w1*preds_val_rf+w2*preds_val_xgb+w3*preds_val_lgb)/3.0
    return log_loss(y_xgb, preds_combi, eps=1e-15, normalize=True)    

In [53]:
combina_predsej6_2([0.4,0.2])

0.44623437295448226

In [57]:
from scipy.optimize import minimize
res2 = minimize(combina_predsej6_2, [0.4,0.2], method='nelder-mead',options={'xatol': 1e-8, 'disp': True})
print(res2)

Optimization terminated successfully.
         Current function value: 0.443713
         Iterations: 74
         Function evaluations: 147
 final_simplex: (array([[0.53765585, 1.60589581],
       [0.53765584, 1.6058958 ],
       [0.53765584, 1.60589582]]), array([0.44371308, 0.44371308, 0.44371308]))
           fun: 0.4437130847588739
       message: 'Optimization terminated successfully.'
          nfev: 147
           nit: 74
        status: 0
       success: True
             x: array([0.53765585, 1.60589581])


Empeora, nos quedamos con los valores del modelo con los 4

In [61]:
res = minimize(combina_predsej6, [0.4,0.1,0.2], method='nelder-mead',options={'xatol': 1e-8, 'disp': True})
print(res)

w1 = res.x[0]
w2 = res.x[1]
w3 = res.x[2]
w4= 4.0-w1-w2-w3

print(w1)
print(w2)
print(w3)


preds_combi_tst_2 = (w1*preds_test_rf+w2*preds_test_logit+w3*preds_test_xgb+w4*preds_val_lgb)/4.0
output = pd.DataFrame(preds_combi_tst_2,columns=["Class_"+str(i) for i in range(1,10)])
output.insert(loc=0, column='id', value=df_test.id)
output.to_csv('submission_combi_5cv_ej6.csv', index=False)
pd.read_csv('submission_combi_5cv_ej6.csv').head()

Optimization terminated successfully.
         Current function value: 0.443467
         Iterations: 140
         Function evaluations: 269
 final_simplex: (array([[ 0.76139657, -0.06477597,  2.09651298],
       [ 0.76139657, -0.06477597,  2.09651298],
       [ 0.76139657, -0.06477597,  2.09651298],
       [ 0.76139657, -0.06477597,  2.09651298]]), array([0.44346735, 0.44346735, 0.44346735, 0.44346735]))
           fun: 0.44346735101107543
       message: 'Optimization terminated successfully.'
          nfev: 269
           nit: 140
        status: 0
       success: True
             x: array([ 0.76139657, -0.06477597,  2.09651298])
0.7613965736406751
-0.06477596690165194
2.096512982665669


ValueError: ignored

# ENTREGA

**Todos los pasos realizados deberán ser explicados con detalle en un documento PDF. El nombre del archivo deberá incluir lo siguiente 'PROYECTO3_NombreDelAlumnoSinAcentos.PDF'. Ejemplo: PROYECTO3_PEPITO_PEREZ.PDF**

Si usas Colab, recuerde guardar sus cambios en su repositorio utilizando la opción "Save a copy in GitHub..." del menú File.